In [1]:
#import libraries

import scipy.sparse as sps
import numpy as np
import os
import pandas as pd 

URM_all= sps.load_npz("./Processed Matrices/URM_simple_coo_2020.npz").tocsr()
ICM_all= sps.load_npz("./Processed Matrices/ICM_simple_coo_2020.npz").tocsr()

In [2]:
%load_ext Cython

In [3]:
from Notebooks_utils.evaluation_function import evaluate_algorithm
from Base.Similarity.Compute_Similarity_Python import Compute_Similarity_Python
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

np.random.seed(41148)

In [4]:
from MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython

In [5]:
recommender= MatrixFactorization_BPR_Cython(URM_all)

MatrixFactorization_BPR_Cython_Recommender: URM Detected 1079 (4.15 %) cold items.


In [6]:
recommender.fit(learning_rate = 0.1, epochs=720, num_factors=200, batch_size=1, positive_reg=0.01, negative_reg=1e-05, sgd_mode="sgd")

MF_BPR: Processed 7947 ( 99.99% ) in 0.60 seconds. BPR loss 4.28E-02. Sample per second: 13314
MF_BPR: Epoch 1 of 720. Elapsed time 0.19 sec
MF_BPR: Processed 7947 ( 99.99% ) in 0.76 seconds. BPR loss 5.71E-02. Sample per second: 10448
MF_BPR: Epoch 2 of 720. Elapsed time 0.36 sec
MF_BPR: Processed 7947 ( 99.99% ) in 0.93 seconds. BPR loss 7.57E-02. Sample per second: 8513
MF_BPR: Epoch 3 of 720. Elapsed time 0.53 sec
MF_BPR: Processed 7947 ( 99.99% ) in 1.09 seconds. BPR loss 9.82E-02. Sample per second: 7281
MF_BPR: Epoch 4 of 720. Elapsed time 0.69 sec
MF_BPR: Processed 7947 ( 99.99% ) in 0.25 seconds. BPR loss 1.15E-01. Sample per second: 31810
MF_BPR: Epoch 5 of 720. Elapsed time 0.84 sec
MF_BPR: Processed 7947 ( 99.99% ) in 0.41 seconds. BPR loss 1.47E-01. Sample per second: 19459
MF_BPR: Epoch 6 of 720. Elapsed time 1.00 sec
MF_BPR: Processed 7947 ( 99.99% ) in 0.57 seconds. BPR loss 1.79E-01. Sample per second: 13960
MF_BPR: Epoch 7 of 720. Elapsed time 1.16 sec
MF_BPR: Process

In [7]:
rec_df=pd.DataFrame({"user_id":[],"item_list":[]})
target_user_list=list(pd.read_csv("./Data/2020/data_target_users_test.csv").user_id)

for uid in target_user_list:
    rec_list=map(str,recommender.recommend(uid)[:10])
    rec_df=rec_df.append({"user_id":uid,"item_list":' '.join(rec_list)},ignore_index=True)

    
rec_df=rec_df.astype({"user_id":'int32',"item_list":'str'})
rec_df=rec_df.set_index('user_id')
rec_df.to_csv("MF_BPR_recommendations.csv")